In [46]:
import os
import pandas as pd
import numpy as np


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model


from mlxtend.classifier import EnsembleVoteClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_predict

import warnings
warnings.simplefilter('ignore')

In [47]:
cwd = os.getcwd()

df_train = pd.read_csv(os.path.join(cwd, 'Preprocessed_Train.csv')) # preprocessed train data
df_test = pd.read_csv(os.path.join(cwd, 'Preprocessed_Test.csv')) # preprocessed test data


In [48]:
df_id = df_test['id']

In [49]:
X_train, y_train = df_train['comments'], df_train['subreddits']

In [50]:
df_test = df_test['comments']

In [51]:
df_not = pd.read_csv(os.path.join(cwd, 'reddit_train.csv'))  #not preprocessed train data

In [52]:
X_train_not, y_train_not = df_not['comments'], df_not['subreddits']

In [53]:
df_test_not = pd.read_csv(os.path.join(cwd, 'reddit_test.csv'))  #not preprocessed test data

In [54]:
df_test_not = df_test_not['comments']

#### TFIDF VECTORIZER

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=None, smooth_idf=True, sublinear_tf=True)

Encoder = LabelEncoder()

y_train = Encoder.fit_transform(y_train)
X_train = tfidf_vectorizer.fit_transform(X_train)

In [56]:
df_test = tfidf_vectorizer.transform(df_test) # preprocessed test set
df_test_not = tfidf_vectorizer.transform(df_test_not) #not preprocessed test set

In [57]:
y_train_not = Encoder.fit_transform(y_train_not)
X_train_not = tfidf_vectorizer.fit_transform(X_train_not)

In [58]:
clf1 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=1000)
clf2 = linear_model.SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, n_iter=1000, random_state=None, learning_rate='optimal', early_stopping=False, validation_fraction=0.1, n_iter_no_change=5)
clf3 = naive_bayes.MultinomialNB(alpha=0.2, fit_prior=True, class_prior=None)


In [59]:
clf1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [38]:
#higher accuracy
clf1.fit(X_train_not, y_train_not)

#clf1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [39]:
clf2.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [40]:
clf3.fit(X_train, y_train)

MultinomialNB(alpha=0.2, class_prior=None, fit_prior=True)

In [41]:
eclf = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3], weights=[1,1,1],voting='hard')


In [42]:
score = model_selection.cross_val_score(eclf, X_train , y_train, cv=5,scoring='accuracy')
    

In [43]:
print(score)
print(score.mean())

[0.56935714 0.57614286 0.57185714 0.56907143 0.56968355]
0.5712224240506976


In [44]:
score_not = model_selection.cross_val_score(eclf, X_train_not, y_train_not, cv=5, scoring='accuracy')

In [45]:
print(score_not)
print(score_not.mean())

[0.57635714 0.57771429 0.57528571 0.57057143 0.57311237]
0.5746081873195024


## Highest achieved accuracy 

## Create a csv file

In [130]:
df_test = pd.read_csv(os.path.join(cwd, 'Preprocessed_Test.csv'))
columns_new = ['id', 'category']
df_submit = pd.DataFrame(df_test, columns=columns_new)

df_preds = pd.DataFrame(eclf_pred)
df_preds['category'] = eclf_pred

df_submit['category']= df_preds['category']
df_submit['id'] = df_id

In [129]:
df_submit.to_csv('C:/Users/adamb/Desktop/COMP551 Applied Machine Learning/Project2/code/try_October15__Try.csv', index=False)